In [18]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# import nltk
# nltk.download('stopwords')
from nltk_utils import bag_of_words, tokenize, stem
from nltk.corpus import stopwords
from model import NeuralNet

with open('data.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
stop_words = list(set(stopwords.words('english')))
punctuation = ['?', '.', '!',',']
words_excluded = stop_words + punctuation
all_words = [stem(w) for w in all_words if w not in words_excluded]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')


20 patterns
4 tags: ['Lamda', 'futsal', 'marchingband', 'tdc']
64 unique stemmed words: ['(', ')', '2013/2014', 'agung', 'ani', 'art', 'basketbal', 'chairman', 'club', 'coach', 'competit', 'council', 'court', 'cup', 'danc', 'energ', 'ever', 'exactli', 'fawwaz', 'field', 'fikri', 'flag', 'fls2p', 'girl', 'go', 'heard', 'hirziah', 'i', 'ipan', 'is', 'join', 'know', 'learn', 'like', 'male', 'mardin', 'meet', 'member', 'mht', 'mrs.', 'music', 'olympiad', 'open', 'other', 'perform', 'play', 'pretti', 'rahmat', 'respons', 'saman', 'see', 'sexi', 'student', 'team', 'tenni', 'thamrin', 'theatr', 'toc', 'train', 'trumpet', 'use', 'what', 'where', 'win']
64 4
Epoch [100/1000], Loss: 0.2698
Epoch [200/1000], Loss: 0.0599
Epoch [300/1000], Loss: 0.0085
Epoch [400/1000], Loss: 0.0038
Epoch [500/1000], Loss: 0.0017
Epoch [600/1000], Loss: 0.0018
Epoch [700/1000], Loss: 0.0015
Epoch [800/1000], Loss: 0.0008
Epoch [900/1000], Loss: 0.0005
Epoch [1000/1000], Loss: 0.0006
final loss: 0.0006
training com

In [19]:
print(stop_words)

['for', 'below', 'this', 'again', 'y', "didn't", 'do', 'into', 'should', 'won', 'there', 'didn', 'myself', 'ourselves', 'they', 'been', 'on', 'other', 'no', 'hadn', 'shan', 'them', 'mustn', 'couldn', "doesn't", 'yourselves', 'the', 'hers', 'themselves', 'was', 've', 'these', 'if', 'such', "shan't", 'she', 'further', 'its', 'weren', "she's", 's', 'same', 'each', 'until', 'what', "mustn't", 'are', 'needn', 'me', "wouldn't", 'off', 'd', 'were', 'here', 'am', 'as', 'in', 'with', 'own', 'we', 'all', 'very', "weren't", 'it', "you're", 'm', 'that', 'doesn', 'which', 'their', "aren't", 'over', 'your', 'and', 'or', 'not', 'theirs', "hadn't", 'too', "it's", 'now', 'had', "mightn't", 'against', 'because', "hasn't", 'down', 'doing', 'to', "that'll", 'at', 'don', "haven't", 'under', 'being', 'mightn', 'through', 'few', 'whom', 'both', 'about', 'yourself', 'between', 'll', 'up', "you've", 'than', 'but', 'wasn', 'himself', 'her', 'by', 'yours', 'wouldn', 'you', 'o', 'have', 're', 'while', 'be', 'does

In [8]:
len(X_train)

20

In [9]:
xy

[(['Any',
   'other',
   'club',
   'than',
   'basketball',
   'that',
   'use',
   'this',
   'basketball',
   'court',
   'field',
   'for',
   'training',
   '?'],
  'futsal'),
 (['I',
   'heard',
   'coach',
   'Agung',
   'trained',
   'a',
   'team',
   'here',
   ',',
   'what',
   'club',
   'is',
   'it',
   'exactly',
   '?'],
  'futsal'),
 (['What',
   'club',
   'fawwaz',
   'joined',
   'when',
   'he',
   'was',
   'a',
   'student',
   '?'],
  'futsal'),
 (['Is',
   'there',
   'any',
   'club',
   'ever',
   'used',
   'the',
   'tennis',
   'court',
   'for',
   'training',
   '?'],
  'futsal'),
 (['What', 'club', 'that', 'all', 'member', 'are', 'male', '?'], 'futsal'),
 (['I',
   'like',
   'to',
   'play',
   'music',
   ',',
   'do',
   'you',
   'know',
   'what',
   'club',
   'should',
   'I',
   'go',
   'with',
   '?'],
  'marchingband'),
 (['Where', 'can', 'I', 'play', 'trumpet', '?'], 'marchingband'),
 (['What',
   'club',
   'that',
   'under',
   'Mrs.',
 

In [16]:
[1,2,3]+ [4,5,6]

[1, 2, 3, 4, 5, 6]